In [2]:
from rdkit import Chem
from config import MOPACMozymeConfig
# Create a molecule
mol = Chem.MolFromSmiles("C=CC=O")

# Create MOPAC config
config = MOPACMozymeConfig()
config.add_molecule(mol)

# Get the configuration string
print(config.get_config_str())

PDBOUT MOZYME GEO-OK SETPI CHARGE=0
MOPACMozymeConfig 
| Unnamed 
HETATM    1  C1  UNL     1       0.000   0.000   0.000  1.00  0.00           C  
HETATM    2  C2  UNL     1       0.000   0.000   0.000  1.00  0.00           C  
HETATM    3  C3  UNL     1       0.000   0.000   0.000  1.00  0.00           C  
HETATM    4  O1  UNL     1       0.000   0.000   0.000  1.00  0.00           O  

1 2
3 4


In [3]:
from rdkit import Chem
from interface import MOPACInterface, MOPACMozymeConfig

# Create a molecule
mol = Chem.MolFromSmiles("C=CC=O")

# Create interface with Mozyme configuration
interface = MOPACInterface(config=MOPACMozymeConfig())

# Add molecules
interface.update_config(mol)

# Run calculation
out_path, arc_path = interface.run_job(
    base_dir="./mopac_runs",
    debug=True
)

MOPAC input file written: mopac_runs/4XBIXX.mop


          MOPAC Job: "4XBIXX.mop" ended normally on Apr 21, 2025, at 23:33.



In [5]:
import os
from rdkit import Chem
from rdkit.Chem import AllChem
from config import MOPACConfig
from optimizer import MOPACOptimizer
from energy import MOPACEnergyCalculator

def test_mopac_optimization_and_energy():
    # Create a simple molecule (ethanol)
    smi = "CCO"
    mol = Chem.MolFromSmiles(smi)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    
    # Set up optimizer
    opt_config = MOPACConfig()
    opt_config.keywords.extend(["PM7", "PRECISE"])  # Use PM7 with precise optimization
    optimizer = MOPACOptimizer(config=opt_config)
    
    # Run optimization
    print("Running optimization...")
    opt_results = optimizer.run_optimization(mol, debug=True)
    print(f"Initial energy: {opt_results['pre_energy']:.2f} kcal/mol")
    print(f"Final energy: {opt_results['post_energy']:.2f} kcal/mol")
    print(f"Optimized coordinates shape: {opt_results['coordinates'].shape}")
    
    # Set up energy calculator
    energy_config = MOPACConfig()
    energy_config.keywords.extend(["PM7", "1SCF"])  # Single-point calculation
    energy_calc = MOPACEnergyCalculator(config=energy_config)
    
    # Run energy calculation on optimized geometry
    print("\nRunning single-point energy calculation...")
    energy_results = energy_calc.calculate_energy(mol, debug=True)
    print(f"Single-point energy: {energy_results['energy']:.2f} kcal/mol")
    
    # Clean up
    for f in os.listdir('.'):
        if f.endswith(('.mop', '.out', '.arc')):
            os.remove(f)

if __name__ == "__main__":
    test_mopac_optimization_and_energy()

Running optimization...
MOPAC input file written: BQX1GL.mop


          MOPAC Job: "BQX1GL.mop" ended normally on Apr 21, 2025, at 23:41.

Initial energy: -54.56 kcal/mol
Final energy: -57.86 kcal/mol
Optimized coordinates shape: (9, 3)

Running single-point energy calculation...
MOPAC input file written: 62JGS7.mop


          MOPAC Job: "62JGS7.mop" ended normally on Apr 21, 2025, at 23:41.

Single-point energy: -54.56 kcal/mol
